# Graph Search Exercise Notebook

This notebook is given in the first graph search exercise course.

It first defines a lot of helper structures. Once these are defined, the exercise questions follow at the end.


## Imports and Constants

Start by importing the library used throughout the nodebook.

There is a high chance that you need to install at least one the following two packages, if you work locally on your laptop:

```
pip3 install graphviz
sudo apt install libcairo2-dev libjpeg-dev libgif-dev
pip3 install pycairo
```

If you use Google colab, then you likely need to run

```
!apt-get install libcairo2-dev libjpeg-dev libgif-dev
!pip install pycairo
```

**Important**: Do not continue, if you get an error with these imports. They need to be fixed first


In [ ]:
import io
import itertools
import math
import os
import shutil
import subprocess
import time

import cairo
import graphviz
import IPython.display

Next define the following global constants used in the notebook.


In [ ]:
T = True
F = False

# Some color constants. These are rgb values between 0 and 1
white = (1, 1, 1)
black = (0, 0, 0)
red = (0.91, 0.3, 0.24)
green = (0, 0.63, 0)
blue = (0, 0.43, 0.9)
orange = (1, 0.5, 0)
pink = (1, 0.15, 0.60)
yellow = (0.96, 0.82, 0.25)
gray = (0.5, 0.5, 0.5)

## Graphs and Grids

Next, we define three data structures to represent graphs and grids. There is an (unweighted) adjacency list, a weighted adjacency list, and a grid.

All three data structures are used to represent (directed) graphs. Every graph has a list of nodes and (directed) edges.


In [ ]:
class AdjList:
    """Stores an unweighted graph as adjacency list.

    Nodes are denoted by integers from 0 to node_count-1.
    """

    def __init__(self, data):
        """Constructs a weighted or unweighted graph.

        The parameter data is a list of list of integers
        that specify for every node the successors.
        """
        self._data = data

        # The next lines of code just check the validity
        # of the adjacency list.
        node_count = len(data)
        for u in range(node_count):
            succ = data[u]
            for v in succ:
                assert v in range(
                    node_count
                ), f"successor {v} is not an integer or out of bounds"
            assert len(set(succ)) == len(
                succ
            ), f"successor list {succ} has duplicated entries"

    def get_node_set(self):
        """Returns a list of all nodes"""
        return range(len(self._data))

    def get_successor(self):
        return lambda v: self._data[v]

In [ ]:
large_graph = AdjList(
    [
        [1, 2, 3],
        [8, 9],
        [4],
        [5],
        [5],
        [10],
        [5, 7],
        [12],
        [9],
        [8, 7],
        [11],
        [6],
        [13],
        [14],
        [15],
        [16],
        [15, 0],
    ]
)
small_graph = AdjList([[1, 2, 3], [0], [4], [5], [5, 2], [3, 6], [4]])

In [ ]:
class WeightedAdjList:
    """Stores a weighted graph as adjacency list.

    Nodes are denoted by integers from 0 to node_count-1.

    Weights must not be negative
    """

    def __init__(self, data):
        """Constructs a weighted or unweighted graph.

        The parameter data is a list of list of pairs of
        numbers and integers. The numbers specify the weight
        of an edge. The integers are the heads of the edges.
        """
        self._data = data

        # The next lines of code just check the validity
        # of the adjacency list.
        node_count = len(data)
        for u in range(node_count):
            weights = [w for w, _ in data[u]]
            succ = [v for _, v in data[u]]
            for v in succ:
                assert v in range(
                    node_count
                ), f"successor {v} is not an integer or out of bounds"
            for w in weights:
                assert w >= 0, f"weight {w} is not a number of is negative"
            assert len(set(succ)) == len(
                succ
            ), f"successor list {succ} has duplicated entries"

    def get_node_set(self):
        """Returns a list of all nodes"""
        return range(len(self._data))

    def get_successor(self):
        return lambda v: self._data[v]

In [ ]:
weighted_graph = WeightedAdjList(
    [
        [(1, 1), (1, 2), (1, 3)],
        [],
        [(1, 4), (1, 7)],
        [(10, 5)],
        [(1, 5)],
        [(0, 6)],
        [(1, 5)],
        [(2, 0)],
    ]
)

In [ ]:
class Grid:
    """Stores a binary occupancy grid. True means
    that a cell is occupied and False means that it
    is free.

    Cells are identified as a pair of integers (x,y)
    where x is the column and y is the row of a cell.

    The free cells are used as nodes in a graph.
    Multiple successor functions exist. For this reason,
    the successor functions are defined outside of the
    grid class.
    """

    def __init__(self, data):
        self._data = data

        # Check whether data is valid.
        width = self.get_width()
        height = self.get_height()
        for y in range(height):
            assert len(data[y]) == width, (
                f"All rows in a grid must have the same width {width}. "
                + "Row {y} is {len(data[y])} long."
            )
            for x in range(width):
                assert (
                    data[y][x] is True or data[y][x] is False
                ), "All cell elements must be True or False"

    def get_width(self):
        return len(self._data[0])

    def get_height(self):
        return len(self._data)

    def is_free(self, p):
        """Checks for a pair of x,y whether the cell is free.

        Cells outside of the grid are occupied by definition, i.e.,
        returns False for coordinates that are outside of the grid.
        """
        x, y = p
        if x not in range(self.get_width()):
            return False
        elif y not in range(self.get_height()):
            return False
        else:
            return not self._data[y][x]

    def get_free_cell_set(self):
        """Returns a list of all free cell coordinates"""
        return [
            (x, y)
            for x in range(self.get_width())
            for y in range(self.get_height())
            if self.is_free((x, y))
        ]

    def get_occupied_cell_set(self):
        """Returns a list of all occupied cell coordinates"""
        return [
            (x, y)
            for x in range(self.get_width())
            for y in range(self.get_height())
            if not self.is_free((x, y))
        ]

## Example Graphs

Next, we define some example graphs and grids used throughout this notebook.


In [ ]:
maze_grid = Grid(
    [
        [F, F, F, F, F, F, F],
        [F, T, T, T, T, T, F],
        [F, T, F, T, F, F, F],
        [F, T, F, T, F, T, F],
        [T, T, F, F, F, T, F],
        [F, T, F, T, T, T, F],
        [F, T, F, F, F, F, F],
        [F, F, F, F, F, T, F],
    ]
)

In [ ]:
center_block_grid = Grid(
    [
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
        [F, F, T, T, T, F, F],
        [F, F, T, T, T, F, F],
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
    ]
)

In [ ]:
wall_grid = Grid(
    [
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
        [T, T, T, T, T, T, T],
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F],
    ]
)

In [ ]:
corner_grid = Grid(
    [
        [T, T, T, F, F, F, F, F, F, F],
        [T, T, T, F, F, F, F, F, F, F],
        [T, T, T, F, F, F, F, F, F, F],
        [T, T, T, F, F, F, F, F, F, F],
        [T, T, T, F, F, F, F, F, F, F],
        [T, T, T, F, F, F, F, F, F, F],
        [F, F, F, F, F, F, F, F, F, F],
        [T, T, T, F, T, T, T, T, T, T],
        [T, T, T, F, T, T, T, T, T, T],
        [T, T, T, F, T, T, T, T, T, T],
    ]
)

## Visualization

Next, we define functions to help visualize graphs and grids.

The implementation of these functions is not of interest to the motion planning course. You can consider them as black boxes.


In [ ]:
def format_graphviz_color(color):
    r, g, b = color
    return f"#{int(255*r):02x}{int(255*g):02x}{int(255*b):02x}"

In [ ]:
def show_graph(node_set, successor, node_color=None, edge_color=None, pdffile=None):
    # set default values
    if node_color is None:
        node_color = {}
    if edge_color is None:
        edge_color = {}

    # assert input is valid
    node_set = set(node_set)
    for tail in node_set:
        for head in successor(tail):
            assert head in node_set, (tail, head, node_set)
    for v in node_color.keys():
        assert v in node_set, (v, node_set)
    for tail, head in edge_color.keys():
        assert tail in node_set, (tail, head, node_set)
        assert head in node_set, (tail, head, node_set)

    # actually render something
    dot = graphviz.Digraph(engine="neato")
    node_list = sorted(x for x in node_set)
    for x in node_list:
        dot.node(
            name=str(x),
            color="black",
            fillcolor=format_graphviz_color(node_color.get(x, white)),
            style="filled",
        )
    for x in node_list:
        for y in sorted(successor(x)):
            if (x, y) in edge_color:
                dot.edge(
                    tail_name=str(x),
                    head_name=str(y),
                    color=format_graphviz_color(edge_color[x, y]),
                    penwidth="3",
                )
            else:
                dot.edge(tail_name=str(x), head_name=str(y))

    # output render result
    if pdffile is not None:
        if pdffile.endswith(".pdf"):
            pdffile = pdffile[:-4]
        dot.render(filename=pdffile, cleanup=True, format="pdf")
    return dot

In [ ]:
show_graph(
    node_set=large_graph.get_node_set(),
    successor=large_graph.get_successor(),
    node_color={5: orange, 13: blue},
    edge_color={(1, 9): green},
)

In [ ]:
def show_weighted_graph(
    node_set, successor, node_color=None, edge_color=None, pdffile=None
):
    # set default values
    if node_color is None:
        node_color = {}
    if edge_color is None:
        edge_color = {}

    # assert input is valid
    node_set = set(node_set)
    for tail in node_set:
        for weight, head in successor(tail):
            assert 0 <= weight, weight
            assert head in node_set, (head, node_set)
    for v in node_color.keys():
        assert v in node_set, (v, node_set)
    for tail, head in edge_color.keys():
        assert tail in node_set, (tail, head, node_set)
        assert head in node_set, (tail, head, node_set)

    # actually render something
    dot = graphviz.Digraph(engine="neato")
    node_list = sorted(x for x in node_set)
    for x in node_list:
        dot.node(
            name=str(x),
            color="black",
            fillcolor=format_graphviz_color(node_color.get(x, white)),
            style="filled",
        )
    for x in node_list:
        for weight, y in sorted(successor(x)):
            if (x, y) in edge_color:
                dot.edge(
                    tail_name=str(x),
                    head_name=str(y),
                    color=format_graphviz_color(edge_color[x, y]),
                    penwidth="3",
                    label=f"{weight:.1f}",
                )
            else:
                dot.edge(tail_name=str(x), head_name=str(y), label=f"{weight:.1f}")

    if pdffile is not None:
        if pdffile.endswith(".pdf"):
            pdffile = pdffile[:-4]
        dot.render(filename=pdffile, cleanup=True, format="pdf")
    return dot

In [ ]:
show_weighted_graph(
    node_set=weighted_graph.get_node_set(), successor=weighted_graph.get_successor()
)

In [ ]:
def show_grid(grid, node_color=None, pngfile=None, svgfile=None, pdffile=None):
    # Use {} as default value. See
    # https://dollardhingra.com/blog/python-mutable-default-arguments/
    # for details
    if node_color is None:
        node_color = {}

    # Fill occupied cells with black color if nothing else
    # is requested.
    for p in grid.get_occupied_cell_set():
        if p not in node_color:
            node_color[p] = (0, 0, 0)

    # Setup some constants and figure out the canvas size
    grid_width = grid.get_width()
    grid_height = grid.get_height()
    hlabel_space = 40
    vlabel_space = 40
    cell_pixel_width = 30
    canvas_width = hlabel_space + (grid_width + 1) * cell_pixel_width
    canvas_height = vlabel_space + (grid_height + 1) * cell_pixel_width

    # Function that does the actual drawing
    def draw(surface):
        context = cairo.Context(surface)

        context.set_source_rgb(1, 1, 1)
        context.rectangle(0, 0, canvas_width, canvas_height)
        context.fill()

        context.translate(hlabel_space, vlabel_space)
        for x in range(grid_width):
            for y in range(grid_height):
                r, g, b = node_color.get((x, y), (1, 1, 1))

                rect_x = x * cell_pixel_width
                rect_y = y * cell_pixel_width

                context.set_source_rgb(r, g, b)
                context.rectangle(rect_x, rect_y, cell_pixel_width, cell_pixel_width)
                context.fill()

                context.set_source_rgb(0, 0, 0)
                context.rectangle(rect_x, rect_y, cell_pixel_width, cell_pixel_width)
                context.stroke()

        context.set_font_size(20)

        for y in range(grid_height + 1):
            text = str(y)
            if y == grid_height:
                text = "y"

            text_width = context.text_extents(text).width
            context.move_to(-8 - text_width, y * cell_pixel_width + 22)
            context.text_path(text)
            context.stroke()

        context.rotate(-math.pi / 2)

        for x in range(grid_width + 1):
            text = str(x)
            if x == grid_width:
                text = "x"

            context.move_to(8, x * cell_pixel_width + 22)
            context.text_path(text)
            context.stroke()

    # Setup a cairo surface that writes to an in-memory file
    svg_buffer = io.BytesIO()
    surface = cairo.SVGSurface(svg_buffer, canvas_width, canvas_height)
    draw(surface)
    if pngfile is not None:
        surface.write_to_png(pngfile)
    surface.finish()

    svg_data = svg_buffer.getvalue()
    if svgfile is not None:
        with open(svgfile, "wb") as out:
            out.write(svg_data)

    if pdffile is not None:
        pdf_buffer = io.BytesIO()
        with cairo.PDFSurface(pdf_buffer, canvas_width, canvas_height) as surface:
            draw(surface)
        with open(pdffile, "wb") as out:
            out.write(pdf_buffer.getvalue())

    return IPython.display.SVG(data=svg_data)

In [ ]:
show_grid(grid=maze_grid, node_color={(6, 4): orange, (2, 2): blue})

## Queue Data Structures

Next, the FIFO and priority queues are defined. You can look at their implementation, if you are interested. However, how to implement them is not subject of this course.


In [ ]:
class FirstInFirstOutQueue:
    """A queue where the first element pushed is the first one popped"""

    def __init__(self, elements=None):
        """Creates a queue with potentially some initial queue elements.

        q = FirstInFirstOutQueue(elements)

        is equivalent to

        q = FirstInFirstOutQueue()
        for e in elements:
            q.push(e)
        """
        if elements is not None:
            self._data = elements + [None]
            self._begin = 0
            self._end = len(elements)
        else:
            self._data = [None]
            self._begin = 0
            self._end = 0

    def __len__(self):
        """Returns number of elements in the queue"""
        if self._begin <= self._end:
            return self._end - self._begin
        else:
            return len(self._data) + self._end - self._begin

    def __iter__(self):
        """Iterates over all elements in the queue in the
        order in which they would be popped
        """
        if self._begin <= self._end:
            return iter(self._data[self._begin : self._end])
        else:
            return itertools.chain(
                iter(self._data[self._begin :]), iter(self._data[: self._end])
            )

    def __repr__(self):
        return f'FirstInFirstOutQueue([{", ".join((str(x) for x in self))}])'

    def push(self, element):
        """Add an element to the end of the queue."""
        if len(self) == len(self._data) - 1:
            self._reallocate_buffer()

        index = self._end
        self._data[index] = element
        self._end = self._advance(self._end)
        assert self._end != self._begin

    def pop(self):
        """Remove an element from the front of the queue and return it.

        Queue must not be empty when calling pop."""
        assert len(self) != 0
        result = self._data[self._begin]
        self._begin = self._advance(self._begin)
        return result

    def _advance(self, pos):
        if pos == len(self._data) - 1:
            return 0
        else:
            return pos + 1

    def _reallocate_buffer(self):
        if self._begin <= self._end:
            new_buffer = self._data[self._begin : self._end]
        else:
            new_buffer = self._data[self._begin :] + self._data[: self._end]

        new_buffer = new_buffer + [None] * (len(self._data) + 1)
        new_end = len(self)

        self._data = new_buffer
        self._begin = 0
        self._end = new_end

In [ ]:
q = FirstInFirstOutQueue()
q.push(3)
q.push(42)
q.push(121)
assert q.pop() == 3
assert q.pop() == 42
q.push(86)
assert q.pop() == 121
assert q.pop() == 86
assert len(q) == 0

The implementation of a priority queue is not straight forward. If you are interested in the algorithmic details search for "Binary Heap".


In [ ]:
class PriorityQueue:
    """A minimum priority queue.

    The queue contains pairs of (key, element). Each pop operation
    removes the element with the smallest key from the queue.
    If two elements have the same key, then the elements are compared
    and the smaller element is popped first.

    The queue cannot contain the equal elements multiple time with
    varying keys.
    """

    def __init__(self, elements=None):
        if elements is not None:
            self._heap = elements
            self._pos = {element: i for i, (key, element) in enumerate(elements)}

            for i in range(len(elements) - 1, -1, -1):
                self._sift_down(i)
        else:
            self._heap = []
            self._pos = {}

    def __len__(self):
        return len(self._heap)

    def __iter__(self):
        """Iterats over all (key,element) pairs in the queue in
        an unspecified order"""
        return iter(self._heap)

    def __repr__(self):
        return f"PriorityQueue({self._heap})"

    def push_or_decrease_key(self, element, key):
        """Adds the pair (key, element) to the queue, if element is
        not yet part of the queue. Otherwise the key of element is set
        to the minimum of the already stored key or the key parameter"""
        if element not in self._pos:
            index = len(self)
            self._heap.append((key, element))
            self._pos[element] = index
            self._sift_up(index)
        else:
            index = self._pos[element]
            old_key, old_element = self._heap[index]
            assert element == old_element
            if key < old_key:
                self._heap[index] = (key, element)
                self._sift_up(index)

    def pop(self):
        """Removes the element with the minimum key from the queue
        and returns the element. If there are multiple elements with
        equal minimum keys, then the smallest element is returned first.

        pop can only be called if the queue is not empty"""
        assert len(self) != 0

        if len(self) == 1:
            _, result = self._heap[0]
            self._heap = []
            self._pos = {}
            return result
        else:
            assert len(self) > 1
            self._swap(0, len(self) - 1)
            _, result = self._heap.pop()
            self._pos.pop(result)
            self._sift_down(0)
            return result

    def _swap(self, i, j):
        assert 0 <= i and i < len(self)
        assert 0 <= j and j < len(self)

        key_i, element_i = self._heap[i]
        key_j, element_j = self._heap[j]

        self._heap[i] = (key_j, element_j)
        self._heap[j] = (key_i, element_i)

        self._pos[element_i] = j
        self._pos[element_j] = i

    def _sift_up(self, node):
        while node != 0:
            assert 0 <= node and node < len(self)

            parent = (node - 1) // 2
            if self._heap[parent] < self._heap[node]:
                return
            else:
                self._swap(parent, node)
                node = parent

    def _sift_down(self, node):
        while True:
            assert 0 <= node and node < len(self)
            # Compute child indices
            left_child = 2 * node + 1
            right_child = 2 * node + 2

            # Figure out what the smallest child is
            has_no_child = left_child >= len(self)
            only_has_left_child = right_child == len(self)
            has_both_children = right_child < len(self)

            if has_no_child:
                return
            elif only_has_left_child:
                smallest_child = left_child
            else:
                assert has_both_children
                smallest_child = (
                    left_child
                    if self._heap[left_child] < self._heap[right_child]
                    else right_child
                )

            # Swap places with smallest child if necessary
            if self._heap[smallest_child] > self._heap[node]:
                return
            else:
                self._swap(smallest_child, node)
                node = smallest_child

In [ ]:
q = PriorityQueue()
q.push_or_decrease_key(3, 2.0)
q.push_or_decrease_key(42, 10.0)
q.push_or_decrease_key(121, -1.0)
assert q.pop() == 121
assert q.pop() == 3
q.push_or_decrease_key(86, 5.0)
assert q.pop() == 86
assert q.pop() == 42
assert len(q) == 0

In [ ]:
q = PriorityQueue()
q.push_or_decrease_key(3, 10.0)
q.push_or_decrease_key(42, 9.0)
q.push_or_decrease_key(3, 8.0)
assert q.pop() == 3
assert q.pop() == 42
assert len(q) == 0

In [ ]:
q = PriorityQueue()
q.push_or_decrease_key(3, 9.0)
q.push_or_decrease_key(42, 10.0)
q.push_or_decrease_key(3, 11.0)
assert q.pop() == 3
assert q.pop() == 42
assert len(q) == 0

# Exercises


Now that all necessary definitions are done, the exercises can follow.


## Exercise 1: Implement Breath First Search


In [ ]:
from typing import List


def bfs(successor, source, target) -> list:
    """Returns a list of node ids that form a path from source
    to target or None if there is no path"""

    q = FirstInFirstOutQueue()

    back_ptr = {}

    curr = source

    while curr != target:
        for x in successor(curr):
            if x not in back_ptr:
                back_ptr[x] = curr
                q.push(x)

        if len(q) <= 0:
            return None
        curr = q.pop()

    path = [target]
    curr = target
    while curr != source:
        curr = back_ptr[curr]
        path.append(curr)

    return list(reversed(path))

In [ ]:
assert bfs(large_graph.get_successor(), 0, 6) == [0, 3, 5, 10, 11, 6]
assert bfs(large_graph.get_successor(), 0, 9) == [0, 1, 9]
assert bfs(large_graph.get_successor(), 0, 8) == [0, 1, 8]
assert bfs(large_graph.get_successor(), 12, 12) == [12]
assert bfs(large_graph.get_successor(), 12, 7) == [12, 13, 14, 15, 16, 0, 1, 9, 7]
assert bfs(small_graph.get_successor(), 5, 0) is None

## Exercise 2: Implement Manhattan Motion Model


In [ ]:
from typing import Tuple


def compute_manhattan_successor_list(grid: Grid, tail: tuple[int, int]):
    """Returns list of unweighted grid successors
    that can move horizontally and vertically.
    """
    x, y = tail
    north = (x, y + 1)
    south = (x, y - 1)
    east = (x + 1, y)
    west = (x - 1, y)

    directions = [north, east, south, west]

    return [d for d in directions if grid.is_free(d)]


def manhattan_successor(grid):
    return lambda tail: compute_manhattan_successor_list(grid, tail)

In [ ]:
assert bfs(manhattan_successor(maze_grid), (0, 0), (2, 2)) == [
    (0, 0),
    (1, 0),
    (2, 0),
    (3, 0),
    (4, 0),
    (5, 0),
    (6, 0),
    (6, 1),
    (6, 2),
    (5, 2),
    (4, 2),
    (4, 3),
    (4, 4),
    (3, 4),
    (2, 4),
    (2, 3),
    (2, 2),
]
assert bfs(manhattan_successor(maze_grid), (0, 5), (3, 6)) == [
    (0, 5),
    (0, 6),
    (0, 7),
    (1, 7),
    (2, 7),
    (3, 7),
    (3, 6),
]
assert bfs(manhattan_successor(maze_grid), (3, 4), (6, 6)) == [
    (3, 4),
    (2, 4),
    (2, 5),
    (2, 6),
    (3, 6),
    (4, 6),
    (5, 6),
    (6, 6),
]

## Exercise 3: Implement Dijkstra's Algorithm


In [ ]:
def dijkstra(successor, source, target) -> tuple[int, list]:
    """Searches for a shortest path from source to target. Shortest means
    that the sum over the edge weights in the path is minimum.

    Returns the pair (dist, path) if a path is found. dist is the sum
    of the edge weights in the path and path is the sequence of nodes.

    If no path exists, returns (None, None)
    """
    q = PriorityQueue()

    back_ptr = {}
    tentative_dist = {source: 0}

    curr = source

    while curr != target:
        for dist, node in successor(curr):
            tentative = tentative_dist[curr] + dist
            if node not in back_ptr or tentative < tentative_dist[node]:
                back_ptr[node] = curr
                tentative_dist[node] = tentative
                q.push_or_decrease_key(node, tentative)

        if len(q) <= 0:
            return None, None

        curr = q.pop()

    path = [target]
    curr = target
    while curr != source:
        curr = back_ptr[curr]
        path.append(curr)

    return tentative_dist[target], list(reversed(path))

In [ ]:
assert dijkstra(weighted_graph.get_successor(), 0, 5) == (3, [0, 2, 4, 5])
assert dijkstra(weighted_graph.get_successor(), 0, 6) == (3, [0, 2, 4, 5, 6])
assert dijkstra(weighted_graph.get_successor(), 2, 3) == (4, [2, 7, 0, 3])
assert dijkstra(weighted_graph.get_successor(), 3, 6) == (10, [3, 5, 6])
assert dijkstra(weighted_graph.get_successor(), 7, 0) == (2, [7, 0])
assert dijkstra(weighted_graph.get_successor(), 6, 4) == (None, None)

## Exercise 4: Implement King Motion Model


In [ ]:
def compute_king_successor_list(grid, tail):
    """Weighted grid successor that can move like chess king"""
    # Use this way to approximate sqrt(2) to make
    # sure we all have the same approximation
    s2 = math.sqrt(2)

    x, y = tail
    north = (x, y + 1)
    north_east = (x + 1, y + 1)
    east = (x + 1, y)
    south_east = (x + 1, y - 1)
    south = (x, y - 1)
    south_west = (x - 1, y - 1)
    west = (x - 1, y)
    north_west = (x - 1, y + 1)

    directions = [
        (north, 1),
        (north_east, s2),
        (east, 1),
        (south_east, s2),
        (south, 1),
        (south_west, s2),
        (west, 1),
        (north_west, s2),
    ]

    return [(w, d) for d, w in directions if grid.is_free(d)]


def king_successor(grid):
    return lambda tail: compute_king_successor_list(grid, tail)

In [ ]:
assert abs(dijkstra(king_successor(maze_grid), (6, 6), (0, 6))[0] - 6.8) < 0.5
assert abs(dijkstra(king_successor(center_block_grid), (3, 0), (3, 4))[0] - 6.2) < 0.5
assert abs(dijkstra(king_successor(center_block_grid), (3, 0), (6, 7))[0] - 8.2) < 0.5
assert dijkstra(king_successor(wall_grid), (3, 0), (6, 7)) == (None, None)

## Exercise 5: Implement Directional Motion Model


In [ ]:
direction_count = 8
E, NE, N, NW, W, SW, S, SE = range(direction_count)


def is_rect_free(grid, p1, p2):
    """Checks whether all cells in the rectanle defined by two
    coordinate pairs are free.
    """
    x1, y1 = p1
    x2, y2 = p2

    if x1 > x2:
        x1, x2 = x2, x1
    if y1 > y2:
        y1, y2 = y2, y1

    for x in range(x1, x2 + 1):
        for y in range(y1, y2 + 1):
            if not grid.is_free((x, y)):
                return False
    return True


def compute_directional_successor_list(grid, tail):
    """Weighted grid sucessor that can move like a chess king or knight"""
    # Use this way to approximate sqrt(2) and sqrt(5)
    # to make sure we all have the same approximation
    s2 = math.sqrt(2)
    s5 = math.sqrt(5)

    x, y, origin = tail

    delta_list = [
        [(2, -1, NE, s5), (1, 0, E, 1), (2, 1, SE, s5)],  # from E
        [(1, -2, N, s5), (1, -1, NE, s2), (2, -1, E, s5)],  # from NE
        [(-1, -2, NW, s5), (0, -1, N, 1), (1, -2, NE, s5)],  # from N
        [(-1, -2, N, s5), (-1, -1, NW, s2), (-2, -1, W, s5)],  # from NW
        [(-2, -1, NW, s5), (-1, 0, W, 1), (-2, 1, SW, s5)],  # from W
        [(-1, 2, S, s5), (-1, 1, SW, s2), (-2, 1, W, s5)],  # from SW
        [(-1, 2, SW, s5), (0, 1, S, 1), (1, 2, SE, s5)],  # from S
        [(1, 2, S, s5), (1, 1, SE, s2), (2, 1, E, s5)],  # from SE
    ]

    return [
        (w, (x + dx, y + dy, dest))
        for dx, dy, dest, w in delta_list[origin]
        if grid.is_free((x + dx, y + dy))
    ]


def directional_successor(grid):
    return lambda tail: compute_directional_successor_list(grid, tail)

In [ ]:
assert dijkstra(directional_successor(center_block_grid), (0, 7, N), (6, 7, S))[1] == [
    (0, 7, N),
    (1, 5, NE),
    (3, 4, E),
    (5, 5, SE),
    (6, 7, S),
]
assert dijkstra(
    successor=directional_successor(corner_grid), source=(0, 6, E), target=(3, 9, S)
)[1] == [
    (0, 6, 0),
    (1, 6, 0),
    (2, 6, 0),
    (3, 6, 0),
    (4, 6, 0),
    (5, 6, 0),
    (6, 6, 0),
    (8, 5, 1),
    (9, 3, 2),
    (8, 1, 3),
    (6, 0, 4),
    (4, 1, 5),
    (3, 3, 6),
    (3, 4, 6),
    (3, 5, 6),
    (3, 6, 6),
    (3, 7, 6),
    (3, 8, 6),
    (3, 9, 6),
]

# Exercise 6

Design a motion model with a direction state that has a reverse gear. When going forward, the robot can turn left and right. When driving backward it can only go straight.

Come up with a grid, where the resulting path is shorter than when using the directional motion model as presented in the lecture.
